In [2]:
import pandas as pd

In [ ]:
# Load the laptop review dataset
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path=r'C:\Users\Admin\Downloads\flipcart_lapy_data.csv')
docs = loader.load()

In [ ]:
# Show the first document for inspection
docs[0]

Document(metadata={'source': 'flipcart_lapy_data.csv', 'row': 0}, page_content='clean_name: DELL Vostro Intel Core i3 12th Gen 1215U\nram_gb: 8.0\nprocessor: Intel Core i3\nos: Windows 11\nstorage_gb: 512.0\nDisplay Size: 15.6\ncpu_brand: Intel\nweight: 2.97\nbattery_life: 5.2\nprice: 42906\nRating: Not found')

In [ ]:
# Now, let's split the document into smaller chunks for better processing in vector stores
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter= RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=400,
    chunk_overlap=50,
    
)
# Split the documents into chunks
chunks = splitter.split_documents(docs)

In [14]:
print(chunks[0])

page_content='clean_name: DELL Vostro Intel Core i3 12th Gen 1215U
ram_gb: 8.0
processor: Intel Core i3
os: Windows 11
storage_gb: 512.0
Display Size: 15.6
cpu_brand: Intel
weight: 2.97
battery_life: 5.2
price: 42906
Rating: Not found' metadata={'source': 'flipcart_lapy_data.csv', 'row': 0}


In [13]:
len(chunks)

405

In [ ]:
# Load the embedding model that transforms text into vector representations
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load HuggingFace embedding model (all-mpnet-base-v2 is good for sentence embedding)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)


C:\Users\Swami\AppData\Local\Temp\ipykernel_4276\1931618161.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
d:\vs code projects\flipcart chatbot\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\vs code projects\flipcart chatbot\myvenv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently stor

In [16]:
text = chunks[0].page_content
print("Text:\n", text)  

# Get embedding vector for this text
embedding_vector = embedding_model.embed_query(text)

# Show embedding
print("\nEmbedding vector:\n", embedding_vector)

# Optional: Just show first 10 numbers to keep it short
print("\nFirst 10 values:\n", embedding_vector[:10])


Text:
 clean_name: DELL Vostro Intel Core i3 12th Gen 1215U
ram_gb: 8.0
processor: Intel Core i3
os: Windows 11
storage_gb: 512.0
Display Size: 15.6
cpu_brand: Intel
weight: 2.97
battery_life: 5.2
price: 42906
Rating: Not found

Embedding vector:
 [0.004206655081361532, -0.04477154463529587, -0.03329731523990631, -0.0240805484354496, 0.05304639786481857, 0.0072769541293382645, -0.010186158120632172, 0.03840754181146622, 0.0037796017713844776, 0.021179717034101486, -0.04282519593834877, -0.02007475309073925, 0.06284768134355545, 0.07243777811527252, -0.033634718507528305, 0.02828039415180683, 0.028207499533891678, 0.024956481531262398, 0.05109129473567009, 0.014025619253516197, -0.027566146105527878, -0.015269053168594837, 0.03617273271083832, -0.05679996311664581, -0.05151907354593277, -0.017150750383734703, 0.0273432694375515, -0.03674919158220291, 0.004967960063368082, -0.03840804100036621, 0.027146147564053535, -0.03952664136886597, -0.026141680777072906, 0.009206000715494156, 1.694

In [ ]:
# Test a query to perform similarity search on the vector store
query="i want a budget laptop for remote work with good batery life and i5"
result=vectorstore.similarity_search(query)
result[0].page_content

'clean_name: MSI Thin 15 Intel Core i5 12th Gen 12450H\nram_gb: 16.0\nprocessor: Intel Core i5\nos: Windows 11\nstorage_gb: 512.0\nDisplay Size: 15.6\ncpu_brand: Intel\nweight: 1.84\nbattery_life: 7.6\nprice: 54990\nRating: 4.4'

In [ ]:
# Create a chain that uses the LLM to refine queries and answer based on retrieved results
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


In [20]:
llm = Ollama(model="llama3")

C:\Users\Swami\AppData\Local\Temp\ipykernel_4276\2786610211.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [51]:
# template = """Answer the question based only on the following context:

# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)

template = """You are a laptop shopping assistant.
Understand the user request and rewrite it as a clear filter-based requirement for searching laptops.
Always display prices in Indian Rupees (₹), not in USD or any other currency.

User query: {query}
Refined requirement:"""


In [52]:
prompt = PromptTemplate(input_variables=["question"], template=template)
query_chain = LLMChain(llm=llm, prompt=prompt)

In [53]:
user_input = "I want a budget laptop for remote work"
refined_query = query_chain.run(user_input)
print("Refined Query:", refined_query)

Refined Query: Based on your request, I've refined the requirement to help you find the perfect budget-friendly laptop for remote work. Here's what I've come up with:

**Filter-based requirements:**

1. **Price:** ₹30,000 - ₹50,000 (Budget-friendly options)
2. **Processor:** Intel Core i3 or AMD Ryzen 3 (Sufficient processing power for basic tasks like email, word processing, and video conferencing)
3. **RAM:** 4 GB - 8 GB (Adequate memory for multiple applications to run simultaneously)
4. **Storage:** 256 GB - 512 GB SSD (Fast storage for efficient file access and loading times)
5. **Display:** 11.6 inches - 14 inches (Portable and suitable for remote work)
6. **Graphics:** Integrated graphics (No heavy gaming requirements, so integrated graphics should suffice)
7. **Operating System:** Windows 10 or Chrome OS (Popular choices for general productivity tasks)

These filters will help me narrow down the options to find a budget laptop that meets your needs for remote work. Let's get st

In [46]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


In [47]:
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
response = rag_chain.run(refined_query)

In [48]:
print("\nRAG Response:\n", response)


RAG Response:
 Based on the provided information, it seems like you're looking for a budget laptop that meets specific criteria for remote work. Unfortunately, none of the laptops listed meet all the requirements. However, I can try to help you identify some possible options.

The HP laptop has a price tag of $46,390, which is well above your budget of less than $400. The MSI Thin 15 Intel Core i5 12th Gen laptops have prices ranging from $50,990 to $54,990, also exceeding the budget.

To find a suitable option within your budget, you may want to consider looking at other laptop models or brands that meet the specified requirements. You can try searching online marketplaces, retail stores, or manufacturer websites for options that fit your criteria.

Keep in mind that meeting all the requirements simultaneously might be challenging within a budget of less than $400. You might need to prioritize certain features and compromise on others to find an affordable option.

If you'd like me t

In [54]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [57]:
from langchain.agents import Tool
search_tool = Tool(
    name="LaptopSearch",
    func=lambda query: rag_chain.run(query),
    description="Useful for answering questions about laptop features, brands, budgets, etc."
)


from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

In [58]:
response = agent_executor.run("Show me Dell laptops under ₹50000")
print(response)

# Later...
followup = agent_executor.run("Only i5 models please")
print(followup)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LaptopSearch
Action Input: "Show me Dell laptops under ₹50000"
Observation: Based on the provided context, I can help you with that!

The first two DELL laptops have prices below ₹50000:

* clean_name: DELL Intel Core i5 12th Gen 1235U - price: 44990
* clean_name: DELL 15 Intel Core i5 13th Gen 1334U - price: 49750

These two laptops meet the condition of being under ₹50000.
Thought:Thought: Do I need to use a tool? No
AI: It looks like you're interested in Dell laptops under ₹50000. Based on my search results, I found two options that fit your budget. The DELL Intel Core i5 12th Gen 1235U is priced at ₹44,990, and the DELL 15 Intel Core i5 13th Gen 1334U is priced at ₹49,750. Both of these laptops meet your condition of being under ₹50000. Would you like me to provide more information about these laptops or help with a different query?

> Finished chain.
It looks like you're interested in Dell laptop

In [60]:
# View history
for msg in memory.load_memory_variables({})["chat_history"]:
    print(f"{msg.type.upper()}: {msg.content}")

# # Or save to a file
# with open("chat_history.txt", "w") as f:
#     for msg in memory.load_memory_variables({})["chat_history"]:
#         f.write(f"{msg.type.upper()}: {msg.content}\n")


HUMAN: Show me Dell laptops under ₹50000
AI: It looks like you're interested in Dell laptops under ₹50000. Based on my search results, I found two options that fit your budget. The DELL Intel Core i5 12th Gen 1235U is priced at ₹44,990, and the DELL 15 Intel Core i5 13th Gen 1334U is priced at ₹49,750. Both of these laptops meet your condition of being under ₹50000. Would you like me to provide more information about these laptops or help with a different query?
HUMAN: Only i5 models please
AI: Given that you only want i5 models, I can refine the search for you. Based on my previous search results, I found two i5-powered Dell laptops under ₹50000. The DELL Intel Core i5 12th Gen 1235U is priced at ₹44,990, and the DELL XPS 13 Plus Intel Core i5 1244U is priced at ₹49,490. Both of these laptops meet your condition of being under ₹50000 and having an i5 processor. Would you like me to provide more information about these laptops or help with a different query?
